In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import re
import numpy as np
from copy import deepcopy
import random
import scipy.sparse as sp
from time import time

# directory = '../MF/ml-1m'
# ratings = []
# with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
#     for l in f:
#         user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
#         ratings.append({
#             'user_id': user_id,
#             'movie_id': movie_id,
#             'rating': rating,
#             'timestamp': timestamp,
#             })
# ratings = pd.DataFrame(ratings)

# ratings = ratings.drop(['timestamp'],axis=1)
# print(ratings)
# # 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
# df = ratings.sample(frac=1).reset_index(drop=True)
# filter_user = deepcopy(df)
# counts = filter_user['user_id'].value_counts()
# filter_user = filter_user[filter_user['user_id'].isin(counts[counts >= 10].index)]

# filtered_df = deepcopy(filter_user)
# counts = filtered_df['movie_id'].value_counts()
# filtered_df = filtered_df[filtered_df['movie_id'].isin(counts[counts >=10].index)]
# filtered_df = filtered_df.reset_index(drop=True)
# print(filtered_df)


class generate_graph(object):
    def __init__(self, path):
        self.path = path  # '../NGCF-PyTorch/Data/ml-1m'
        train_file = path + '/train.txt'
        
        self.neighbor_dict = {}
        self.user, self.item = [], []
        
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    self.neighbor_dict[int(l[0])] = [int(i) for i in l[1:]]
                    self.user.append(int(l[0]))
        self.user = self.user[:1000]
        
    def jaccard_index(self, u_i, u_j, neighbor_dict):
        u_i_neighbor = self.neighbor_dict[u_i]
        u_j_neighbor = self.neighbor_dict[u_j]
        return len(list(set(u_i_neighbor) & set(u_j_neighbor))) / len(list(set(u_i_neighbor) | set(u_j_neighbor)))

    # 새로운 graph의 node j 에다가 기존 graph의 어떤 node의 neighborhood를 복사할지 zeta에 담기
    def node_copying(self):
        t1 = time()
        zeta = []
        
        for u_j in self.user:
            nor = 0
            zeta_distribution = []
            for u_i in self.user:
                nor+=self.jaccard_index(u_j, u_i, self.neighbor_dict)
            for u_m in self.user:
                zeta_distribution.append(self.jaccard_index(u_j, u_m, self.neighbor_dict) / nor)
            zeta.append(random.choices(self.user, weights=zeta_distribution)[0])
        print('total node copying time cost : ', time() - t1)
        return zeta
    
    def generate_graph(self, epsilon, iteration):
        t2 = time()
        self.epsilon = epsilon
        self.iteration = iteration
        
        generated_node = []
        
        with open(self.path+'/sampled_graph/sampled_graph_'+str(iteration+1), 'w') as f:
            for i in self.user:
                if random.uniform(0,1) < 1-self.epsilon:  # 1-epsilon의 확률로 원래 neighbor 넣기
                    generated_node.append(i)
                else:                                # epsilon의 확률로 zeta에 있는 node의 neighbor로 copy해서 넣기
                    generated_node.append(zeta[i])
                    
                # 만들어지는 새로운 graph를 txt 파일로 저장
                f.write(str(i))
                f.write(' ')
                for j in self.neighbor_dict[generated_node[i]][:-1]:
                    f.write(str(j))
                    f.write(' ')
                f.write(str(self.neighbor_dict[generated_node[i]][-1]))
                f.write('\n')
        print(iteration+1,'-th Graph sampled time cost : ', time() - t2)

In [8]:
t = generate_graph('ml-1m')
zeta = t.node_copying()

zeta
# zeta = node_copying(user)

total node copying time cost :  32.17004346847534


[120,
 908,
 429,
 24,
 277,
 886,
 67,
 201,
 332,
 555,
 795,
 96,
 955,
 421,
 467,
 656,
 250,
 133,
 476,
 495,
 20,
 191,
 397,
 473,
 369,
 298,
 40,
 222,
 455,
 127,
 845,
 625,
 122,
 859,
 207,
 138,
 955,
 571,
 158,
 894,
 563,
 481,
 42,
 592,
 550,
 734,
 46,
 473,
 528,
 230,
 670,
 942,
 396,
 270,
 663,
 145,
 455,
 441,
 58,
 732,
 784,
 224,
 602,
 218,
 954,
 890,
 523,
 466,
 939,
 266,
 457,
 415,
 173,
 810,
 564,
 82,
 934,
 987,
 70,
 725,
 977,
 231,
 505,
 933,
 19,
 21,
 729,
 231,
 215,
 807,
 634,
 875,
 17,
 93,
 577,
 299,
 952,
 483,
 118,
 887,
 748,
 35,
 181,
 227,
 557,
 799,
 606,
 315,
 167,
 918,
 468,
 538,
 272,
 553,
 205,
 551,
 801,
 294,
 577,
 964,
 345,
 695,
 974,
 334,
 557,
 53,
 24,
 909,
 703,
 547,
 273,
 239,
 184,
 523,
 264,
 223,
 880,
 373,
 169,
 517,
 538,
 2,
 162,
 326,
 308,
 700,
 736,
 713,
 962,
 452,
 712,
 345,
 611,
 319,
 730,
 694,
 347,
 112,
 526,
 433,
 924,
 191,
 25,
 376,
 795,
 161,
 326,
 145,
 876,
 887,


In [9]:
# zeta를 한번 만든 후 iteration마다 generate graph하는 방식

for epoch in range(3):
    t.generate_graph(epsilon = 0.01, iteration = epoch)

1 -th Graph sampled time cost :  0.031705617904663086
2 -th Graph sampled time cost :  0.020745277404785156
3 -th Graph sampled time cost :  0.020474910736083984


In [ ]:
# GAT
# 만들어지는 graph마다 embedding 초기화 
# embedding 학습하는 GNN 구조 짜기
# 학습된 embedding 가지고 x_hat (eq.11) 구하고 BPR-OPT를 maximization 시키는 방향으로 학습

In [21]:
# data로 sparse matrix 및 adjacency matrix 만들기

class Data(object):
    def __init__(self, path):
        self.path = path
        
        train_file = path + '/train.txt'
        # path : ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(self.n_users, self.n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/train_adj_mat.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/train_adj_mat.npz', adj_mat)
            
        return adj_mat
    
    
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


In [25]:
# node copying으로 만들어진 graph들에 대해서 adj matrix 만들고 npz 저장하는 과정 

class sampled_graph_to_matrix(object):
    def __init__(self, path, iteration):
        self.path =path
        self.iteration = iteration
        
        sampled_graph = path + '/sampled_graph/sampled_graph_' + str(iteration+1)
        # path : 'ml-1m'
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(sampled_graph, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(self.n_users, self.n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(sampled_graph, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz', adj_mat)
            
        return adj_mat
    
     
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()


In [29]:
for iteration in range(3):
    sampled_graph = sampled_graph_to_matrix(path='ml-1m', iteration = iteration)
    sampled_graph.get_adj_mat()

1000 3953
1000 3953
1000 3953


#### ngcf에서의 w_gc와 w_bi는 gc는 neighbor aggregate하는 부분의 weight, bi는 element-wise product하는 부분의 weight
#### bgcf에서는 single feed forward layer로 구성하였고 총 3개의 weight가 학습됨 (GAT에서 2개)

In [73]:
class GATLayer(nn.Module):
    def __init__(self, n_users, n_items, in_features, out_features, alpha, dropout = 0.2):
        super(GATLayer, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.dropout = dropout
        self.alpha = alpha
        self.in_features = in_features
        self.out_features = out_features
        
        initializer = nn.init.xavier_uniform_
        
        self.W = nn.Parameter(initializer(torch.empty(in_features, out_features)))
        self.embedding_dict = nn.ParameterDict({
            'user_emb' : nn.Parameter(initializer(torch.empty(self.n_users, self.emb_size))),
            'item_emb' : nn.Parameter(initializer(torch.empty(self.n_items, self.emb_size)))
        })
        
        # self.leakyrelu = nn.LeakyReLU(self.alpha)
    
    def forward(self, path, self_feature, iteration):
        self.path = path
        coef = torch.zeros(self.n_users, self.n_items)
        adj_matrix = sp.load_npz(path + '/s_adj_mat_' + str(iteration+1) +'.npz').toarray()
        adj_matrix = torch.Tensor(adj)
        
        user_emb = self.embedding_dict['user_emb']
        item_emb = self.embedding_dict['item_emb']
        
        # target node가 user라고만 가정하고 짠 코드 --> target node가 user건 item이건 되게끔 수정해야함
        # self.W의 크기를 정확하게 확인해야함, 최종적인 각 node의 embedding size 는 1 x emb_size 로 나오게끔
        # Weight가 2개가 필요함, NGCF 논문 참고하여 weight initialization 하나 더 해야함
        score = torch.exp(torch.inner(user_emb, item_emb))
        score = torch.multiply(score, adj_matrix)
        for i in range(score.shape[0]):
            norm = sum(score[i])
            score = score[i] / norm
            coef[i] = score
        
        
        

In [181]:
adj = sp.load_npz('ml-1m/s_adj_mat_1.npz')
adj = adj.toarray()
print(adj.shape)
adj = torch.Tensor(adj)
coef = torch.zeros(3,3953)
user_emb = nn.Parameter(initializer(torch.empty(1000,32)))
item_emb = nn.Parameter(initializer(torch.empty(3953,32)))

x = torch.exp(torch.inner(user_emb, item_emb))
print(x)
x = torch.multiply(x, adj)
print(x)
for i in range(3):
    tot = sum(x[i])
    score = x[i] / tot
    coef[i] = score
    
torch.matmul(coef.T, user_emb[:3]).shape
# x = torch.multiply(x, adj)
# print(x.shape)
# x = F.softmax(x, dim=1)
# x


(1000, 3953)
tensor([[1.0138, 1.0067, 1.0037,  ..., 1.0095, 1.0078, 1.0027],
        [0.9961, 0.9976, 0.9986,  ..., 0.9935, 0.9931, 1.0009],
        [1.0020, 0.9880, 0.9979,  ..., 0.9926, 0.9921, 1.0011],
        ...,
        [0.9969, 0.9958, 0.9989,  ..., 0.9982, 0.9929, 1.0044],
        [0.9958, 0.9918, 1.0055,  ..., 1.0044, 1.0043, 1.0002],
        [0.9938, 1.0044, 1.0072,  ..., 1.0024, 0.9981, 1.0017]],
       grad_fn=<ExpBackward0>)
tensor([[0.0000, 1.0067, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0044, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


torch.Size([3953, 32])

In [81]:
initializer = nn.init.xavier_uniform_
a = nn.Parameter(initializer(torch.empty(5,5)))
a

Parameter containing:
tensor([[ 0.7327, -0.5712,  0.4818, -0.6869,  0.4029],
        [ 0.4169, -0.4035, -0.6392, -0.5619, -0.5682],
        [-0.0924,  0.3034,  0.3748,  0.1871, -0.4546],
        [ 0.3260, -0.5176,  0.3845, -0.0083,  0.1313],
        [-0.1405,  0.0375, -0.5250, -0.3310, -0.7736]], requires_grad=True)

In [82]:
initializer = nn.init.xavier_uniform_
user_emb = nn.Parameter(initializer(torch.empty(5,10)))
user_emb

Parameter containing:
tensor([[-0.4248,  0.3278, -0.4776,  0.1640,  0.5242, -0.2335,  0.1329, -0.1280,
          0.0446,  0.2148],
        [ 0.0805,  0.2520,  0.3278,  0.0761,  0.0423, -0.6018,  0.0667,  0.2250,
          0.5964,  0.2410],
        [-0.4188, -0.5408,  0.5625,  0.0053, -0.3498,  0.3247,  0.0657, -0.0464,
         -0.5074, -0.3182],
        [-0.5155,  0.0474, -0.2445, -0.0620, -0.5623,  0.5452, -0.5621,  0.2565,
          0.3390, -0.5601],
        [-0.4980, -0.2306,  0.2275, -0.5679, -0.1371, -0.5745,  0.2258, -0.1681,
          0.2692,  0.0183]], requires_grad=True)

In [33]:
a = np.matrix([[0,1,2],[1,1,1],[0,0,1]])
adj = np.matrix([[0,0,1],[1,0,0],[1,1,0]])
e = np.matrix([[1,2,3],[4,5,6],[7,8,9]])

print(a,'\n')
print(adj,'\n')
print(e,'\n')

a_jk = np.multiply(a,adj)
print(a_jk, '\n')

np.matmul(a_jk, e)
    
    
    

[[0 1 2]
 [1 1 1]
 [0 0 1]] 

[[0 0 1]
 [1 0 0]
 [1 1 0]] 

[[1 2 3]
 [4 5 6]
 [7 8 9]] 

[[0 0 2]
 [1 0 0]
 [0 0 0]] 



matrix([[14, 16, 18],
        [ 1,  2,  3],
        [ 0,  0,  0]])